# Thư viện 

In [1]:
import sys 
sys.path.append('/kaggle/input/globalstatisticalbasedcode')
import importlib

import os
from enum import Enum
from dataclasses import dataclass

import cv2
import numpy as np
import matplotlib.pyplot as plt
import funcs
importlib.reload(funcs)

<module 'funcs' from '/kaggle/input/globalstatisticalbasedcode/funcs.py'>

# Chạy cho dataset đầu tiên

## Load dataset dùng cho tay trái
URL của dataset trên kaggle:  https://www.kaggle.com/datasets/mahdieizadpanah/birjand-university-mobile-palmprint-databasebmpd <br> 
Dataset gồm các thư mục được đánh số từ 001, 002, .... Mỗi thư mục ứng với 1 người <br>
Trong từng thư mục, tên các file có format sau: "person_cat_side_index" <br> 
Chỉ load dữ liệu cho tay trái tức là side == "L" <br> 
Sau đó tiến hành process cho từng image , nếu trong quá trình gặp lỗi thì bỏ qua <br> 
Chỉ giữ lại person có ít nhất 2 images trở lên để phục vụ cho việc chia thành tập train và test <br> 

In [2]:
data_source = "/kaggle/input/birjand-university-mobile-palmprint-databasebmpd/Birjand University Mobile Palmprint Database (BMPD)"

def parse_person_folder(folderPath):
    features = []
    labels = []

    person = folderPath.split("/")[-1]

    fileNames = os.listdir(folderPath)

    for fileName in fileNames:
        name, ext = os.path.splitext(fileName)
        _, _, side, index = name.split("_")
        
        if side == "L":
            try: 
                feat = funcs.process_img(f"{folderPath}/{fileName}")

                features.append(feat)
                labels.append(int(person))

                print(f"Process file {fileName} ok")
            except Exception as e: 
                print(f"Process file {fileName} error: {e}")
            

    if len(features) < 2:
        raise Exception(f"Load folder {folderPath} error: not have enough 2 samples")

    print(f"Load folder {folderPath} ok")

    return features, labels 

print(f"Start to load dataset from {data_source}")
folderNames = os.listdir(data_source)
features = []
labels = []
for folderName in folderNames:
    try:
        person_features, person_labels = parse_person_folder(f"{data_source}/{folderName}")

        features += person_features
        labels += person_labels
    except Exception as e:
        print(f"{folderName}, error: {e}")

features = np.vstack(features)
labels = np.asarray(labels)

print(f"Shape features: {features.shape}")
print(f"Labels: {labels.shape}")

Start to load dataset from /kaggle/input/birjand-university-mobile-palmprint-databasebmpd/Birjand University Mobile Palmprint Database (BMPD)
Process file 018_F_L_31.JPG ok
Process file 018_F_L_36.JPG ok
Process file 018_S_L_26.JPG error: Not enough valley candidates (need at least 3 after suppression).
Process file 018_F_L_33.JPG ok
Process file 018_F_L_38.JPG ok
Process file 018_F_L_34.JPG ok
Process file 018_S_L_27.JPG ok
Process file 018_S_L_21.JPG ok
Process file 018_S_L_23.JPG error: Computed ROI rectangle is invalid after clamping.
Process file 018_F_L_32.JPG ok
Process file 018_S_L_28.JPG ok
Process file 018_F_L_37.JPG ok
Process file 018_S_L_24.JPG ok
Process file 018_F_L_39.JPG ok
Process file 018_S_L_25.JPG ok
Process file 018_S_L_29.JPG ok
Process file 018_F_L_35.JPG ok
Process file 018_F_L_30.JPG ok
Process file 018_S_L_22.JPG ok
Process file 018_S_L_20.JPG ok
Load folder /kaggle/input/birjand-university-mobile-palmprint-databasebmpd/Birjand University Mobile Palmprint Dat

## Split into train and test dataset
Chia dataset ban đầu thành 1 nửa cho train và 1 nửa cho test

In [3]:
X_train, X_test, Y_train, Y_test = funcs.split_train_test(features, labels, 0.5)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (356, 7)
X_test shape: (356, 7)


## Evaluate on test
Với mỗi sample trong tập test, tiến hành tìm 1 sample trong tập train có distance nhỏ nhất, khi đó kiểm tra label của 2 sample có match với nhau hay không <br> 

In [4]:
funcs.evaluate_on_test(X_train, X_test, Y_train, Y_test)

Accuracy on test: 31.46067415730337 %


# Chạy với dataset của tay bên phải 
Sử dụng dataset trên nhưng dùng tay bên phải tức là side == "R" <br> 

In [5]:
data_source = "/kaggle/input/birjand-university-mobile-palmprint-databasebmpd/Birjand University Mobile Palmprint Database (BMPD)"

def parse_person_folder(folderPath):
    features = []
    labels = []

    person = folderPath.split("/")[-1]

    fileNames = os.listdir(folderPath)

    for fileName in fileNames:
        name, ext = os.path.splitext(fileName)
        _, _, side, index = name.split("_")
        
        if side == "R":
            try: 
                feat = funcs.process_img(f"{folderPath}/{fileName}")

                features.append(feat)
                labels.append(int(person))

                print(f"Process file {fileName} ok")
            except Exception as e: 
                print(f"Process file {fileName} error: {e}")
            

    if len(features) < 2:
        raise Exception(f"Load folder {folderPath} error: not have enough 2 samples")

    print(f"Load folder {folderPath} ok")

    return features, labels 

print(f"Start to load dataset from {data_source}")
folderNames = os.listdir(data_source)
features = []
labels = []
for folderName in folderNames:
    try:
        person_features, person_labels = parse_person_folder(f"{data_source}/{folderName}")

        features += person_features
        labels += person_labels
    except Exception as e:
        print(f"{folderName}, error: {e}")

features = np.vstack(features)
labels = np.asarray(labels)

print(f"Shape features: {features.shape}")
print(f"Labels: {labels.shape}")

X_train, X_test, Y_train, Y_test = funcs.split_train_test(features, labels, 0.5)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

funcs.evaluate_on_test(X_train, X_test, Y_train, Y_test)

Start to load dataset from /kaggle/input/birjand-university-mobile-palmprint-databasebmpd/Birjand University Mobile Palmprint Database (BMPD)
Process file 018_F_R_8.JPG ok
Process file 018_F_R_4.JPG ok
Process file 018_S_R_12.JPG error: Not enough valley candidates (need at least 3 after suppression).
Process file 018_S_R_16.JPG ok
Process file 018_F_R_0.JPG ok
Process file 018_S_R_15.JPG error: Computed ROI rectangle is invalid after clamping.
Process file 018_F_R_3.JPG ok
Process file 018_F_R_5.JPG ok
Process file 018_S_R_18.JPG ok
Process file 018_F_R_7.JPG error: Not enough valley candidates (need at least 3 after suppression).
Process file 018_S_R_17.JPG ok
Process file 018_S_R_10.JPG ok
Process file 018_S_R_11.JPG ok
Process file 018_S_R_14.JPG ok
Process file 018_S_R_19.JPG ok
Process file 018_F_R_6.JPG error: Not enough valley candidates (need at least 3 after suppression).
Process file 018_F_R_1.JPG ok
Process file 018_S_R_13.JPG error: Not enough valley candidates (need at le

# Chạy với dataset khác 
URL của dataset này trên kaggle: https://www.kaggle.com/datasets/mahdieizadpanah/sapienza-university-mobile-palmprint-databasesmpd <br> 
Dữ liệu chỉ chứa tay bên phải <br>

In [6]:
data_source = "/kaggle/input/sapienza-university-mobile-palmprint-databasesmpd/Sapienza University Mobile Palmprint Database(SMPD)"

def parse_person_folder(folderPath):
    features = []
    labels = []

    person = folderPath.split("/")[-1]

    fileNames = os.listdir(folderPath)

    for fileName in fileNames:
        try: 
            feat = funcs.process_img(f"{folderPath}/{fileName}")

            features.append(feat)
            labels.append(int(person))

            print(f"Process file {fileName} ok")
        except Exception as e: 
            print(f"Process file {fileName} error: {e}")
            

    if len(features) < 2:
        raise Exception(f"Load folder {folderPath} error: not have enough 2 samples")

    print(f"Load folder {folderPath} ok")

    return features, labels 

print(f"Start to load dataset from {data_source}")
folderNames = os.listdir(data_source)
features = []
labels = []
for folderName in folderNames:
    try:
        person_features, person_labels = parse_person_folder(f"{data_source}/{folderName}")

        features += person_features
        labels += person_labels
    except Exception as e:
        print(f"{folderName}, error: {e}")

features = np.vstack(features)
labels = np.asarray(labels)

print(f"Shape features: {features.shape}")
print(f"Labels: {labels.shape}")

X_train, X_test, Y_train, Y_test = funcs.split_train_test(features, labels, 0.5)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")

funcs.evaluate_on_test(X_train, X_test, Y_train, Y_test)

Start to load dataset from /kaggle/input/sapienza-university-mobile-palmprint-databasesmpd/Sapienza University Mobile Palmprint Database(SMPD)
Process file 057_F_8.JPG ok
Process file 057_F_2.JPG ok
Process file 057_F_4.JPG ok
Process file 057_P_27.JPG ok
Process file 057_RP_37.JPG ok
Process file 057_RF_19.JPG ok
Process file 057_P_23.JPG ok
Process file 057_RF_14.JPG ok
Process file 057_P_25.JPG ok
Process file 057_F_9.JPG ok
Process file 057_RP_30.JPG ok
Process file 057_RF_18.JPG ok
Process file 057_RF_15.JPG ok
Process file 057_P_21.JPG ok
Process file 057_RP_32.JPG ok
Process file 057_P_28.JPG ok
Process file 057_RF_11.JPG ok
Process file 057_F_6.JPG ok
Process file 057_RP_36.JPG ok
Process file 057_RP_31.JPG ok
Process file 057_F_0.JPG ok
Process file 057_F_3.JPG ok
Process file 057_F_1.JPG ok
Process file 057_P_24.JPG ok
Process file 057_P_22.JPG ok
Process file 057_RP_35.JPG ok
Process file 057_RP_38.JPG ok
Process file 057_RF_10.JPG ok
Process file 057_RP_34.JPG ok
Process fi

Invalid SOS parameters for sequential JPEG


Process file 023_RP_39.JPG error: Not enough valley candidates (need at least 3 after suppression).


Invalid SOS parameters for sequential JPEG


Process file 023_RP_35.JPG error: Computed ROI rectangle is invalid after clamping.


...

Process file 030_RF_14.JPG error: Computed ROI rectangle is invalid after clamping.
Process file 030_F_2.JPG ok


Invalid SOS parameters for sequential JPEG


Process file 030_RP_36.JPG ok


Invalid SOS parameters for sequential JPEG


Process file 030_P_29.JPG error: Computed ROI rectangle is invalid after clamping.
Load folder /kaggle/input/sapienza-university-mobile-palmprint-databasesmpd/Sapienza University Mobile Palmprint Database(SMPD)/030 ok
Process file 085_P_23.JPG ok
Process file 085_F_6.JPG ok
Process file 085_RF_14.JPG ok
Process file 085_F_8.JPG ok
Process file 085_F_2.JPG ok
Process file 085_P_20.JPG ok
Process file 085_P_26.JPG ok
Process file 085_RP_38.JPG error: Computed ROI rectangle is invalid after clamping.
Process file 085_RP_30.JPG ok
Process file 085_RF_17.JPG ok
Process file 085_RP_37.JPG error: Computed ROI rectangle is invalid after clamping.
Process file 085_F_7.JPG ok
Process file 085_RP_33.JPG ok
Process file 085_RF_18.JPG ok
Process file 085_RF_11.JPG ok
Process file 085_RF_12.JPG ok
Process file 085_F_9.JPG ok
Process file 085_RF_10.JPG ok
Process file 085_P_24.JPG ok
Process file 085_RP_36.JPG ok
Process file 085_F_1.JPG ok
Process file 085_RP_39.JPG error: Not enough valley candidat